# packages

In [5]:
pip install psycopg2-binary


Defaulting to user installation because normal site-packages is not writeable
   ---------------------------------------- 0.0/1.2 MB ? eta -:--:--
   ---------------------------------------- 0.0/1.2 MB ? eta -:--:--
   ---------------------------------------- 0.0/1.2 MB ? eta -:--:--
   ---------------------------------------- 0.0/1.2 MB ? eta -:--:--
   ---------------------------------------- 0.0/1.2 MB ? eta -:--:--
   - -------------------------------------- 0.0/1.2 MB 119.1 kB/s eta 0:00:10
   - -------------------------------------- 0.0/1.2 MB 151.3 kB/s eta 0:00:08
   -- ------------------------------------- 0.1/1.2 MB 192.5 kB/s eta 0:00:06
   -- ------------------------------------- 0.1/1.2 MB 196.3 kB/s eta 0:00:06
   -- ------------------------------------- 0.1/1.2 MB 196.3 kB/s eta 0:00:06
   --- ------------------------------------ 0.1/1.2 MB 201.8 kB/s eta 0:00:06
   --- ------------------------------------ 0.1/1.2 MB 226.0 kB/s eta 0:00:05
   ---- -----------------------

In [38]:
import pandas as pd
import psycopg2
from datetime import datetime
from psycopg2.extras import execute_values

## Lecture du fichier csv

In [39]:
df = pd.read_csv("C:/Users/Sylla/Desktop/invoice-2009-03-05 00_00_00.csv", sep = ",")

In [40]:
cols = df.columns
print(cols)
df.head(2)

Index(['InvoiceDate', 'InvoiceId', 'InvoiceLineId', 'TrackId', 'UnitPrice',
       'Quantity', 'BillingAddress', 'BillingCity', 'BillingCountry',
       'BillingPostalCode', 'BillingState', 'CustomerId', 'CustomerFirstName',
       'CustomerLastName', 'CustomerCompany', 'CustomerAddress',
       'CustomerCity', 'CustomerState', 'CustomerCountry', 'CustomerPhone',
       'CustomerEmail', 'CustomerSupportRepId', 'EmployeeId',
       'EmployeeFirstName', 'EmployeeLastName', 'EmployeeBirthDate',
       'EmployeeHireDate', 'EmployeeTitle', 'EmployeeAddress', 'EmployeeCity',
       'EmployeeState', 'EmployeeCountry', 'EmployeePhone', 'EmployeeFax',
       'EmployeeEmail', 'EmployeeReportsTo'],
      dtype='object')


,InvoiceDate,InvoiceId,InvoiceLineId,TrackId,UnitPrice,Quantity,BillingAddress,BillingCity,BillingCountry,BillingPostalCode,...,EmployeeHireDate,EmployeeTitle,EmployeeAddress,EmployeeCity,EmployeeState,EmployeeCountry,EmployeePhone,EmployeeFax,EmployeeEmail,EmployeeReportsTo
0,2009-03-05 00:00:00,16,79,470,0.99,1,801 W 4th Street,Reno,USA,89503,...,2003-10-17 00:00:00,Sales Support Agent,7727B 41 Ave,Calgary,AB,Canada,1 (780) 836-9987,1 (780) 836-9543,steve@chinookcorp.com,2
1,2009-03-05 00:00:00,16,80,472,0.99,1,801 W 4th Street,Reno,USA,89503,...,2003-10-17 00:00:00,Sales Support Agent,7727B 41 Ave,Calgary,AB,Canada,1 (780) 836-9987,1 (780) 836-9543,steve@chinookcorp.com,2


In [41]:
# =========================
# Générer TitleId
# =========================
# Dictionnaire de correspondance entre EmployeeTitle et un identifiant numérique
title_mapping = {title: i+1 for i, title in enumerate(df['EmployeeTitle'].drop_duplicates())}

# Ajouter la colonne TitleId en mappant EmployeeTitle
df['TitleId'] = df['EmployeeTitle'].map(title_mapping).astype(int)

# =========================
# Générer LocationId
# =========================
# On combine les colonnes de localisation (concaténation en une clé unique)
df['LocationKey'] = (
    df['CustomerAddress'].fillna('').astype(str) + '_' +
    df['CustomerCity'].fillna('').astype(str) + '_' +
    df['CustomerState'].fillna('').astype(str) + '_' +
    df['CustomerCountry'].fillna('').astype(str) + '_' +
    df['BillingPostalCode'].fillna('').astype(str)
)

# dictionnaire de correspondance clé → ID numérique
location_mapping = {loc: i+1 for i, loc in enumerate(df['LocationKey'].drop_duplicates())}

# création de LocationId numérique
df['LocationId'] = df['LocationKey'].map(location_mapping).astype(int)

In [29]:
df.head(2)

,InvoiceDate,InvoiceId,InvoiceLineId,TrackId,UnitPrice,Quantity,BillingAddress,BillingCity,BillingCountry,BillingPostalCode,...,EmployeeCity,EmployeeState,EmployeeCountry,EmployeePhone,EmployeeFax,EmployeeEmail,EmployeeReportsTo,TitleId,LocationKey,LocationId
0,2009-03-05 00:00:00,16,79,470,0.99,1,801 W 4th Street,Reno,USA,89503,...,Calgary,AB,Canada,1 (780) 836-9987,1 (780) 836-9543,steve@chinookcorp.com,2,1,801 W 4th Street_Reno_NV_USA_89503,1
1,2009-03-05 00:00:00,16,80,472,0.99,1,801 W 4th Street,Reno,USA,89503,...,Calgary,AB,Canada,1 (780) 836-9987,1 (780) 836-9543,steve@chinookcorp.com,2,1,801 W 4th Street_Reno_NV_USA_89503,1


In [42]:
cols = df.columns
print(cols)

Index(['InvoiceDate', 'InvoiceId', 'InvoiceLineId', 'TrackId', 'UnitPrice',
       'Quantity', 'BillingAddress', 'BillingCity', 'BillingCountry',
       'BillingPostalCode', 'BillingState', 'CustomerId', 'CustomerFirstName',
       'CustomerLastName', 'CustomerCompany', 'CustomerAddress',
       'CustomerCity', 'CustomerState', 'CustomerCountry', 'CustomerPhone',
       'CustomerEmail', 'CustomerSupportRepId', 'EmployeeId',
       'EmployeeFirstName', 'EmployeeLastName', 'EmployeeBirthDate',
       'EmployeeHireDate', 'EmployeeTitle', 'EmployeeAddress', 'EmployeeCity',
       'EmployeeState', 'EmployeeCountry', 'EmployeePhone', 'EmployeeFax',
       'EmployeeEmail', 'EmployeeReportsTo', 'TitleId', 'LocationKey',
       'LocationId'],
      dtype='object')


In [43]:
config = {
    'host': '127.0.0.1',    
    'port': 5432,
    'database': 'dbtest',
    'user': 'sylla',
    'password': 'oumar'       
}

In [44]:
# Connexion à PostgreSQL
conn = psycopg2.connect(**config)
print("Connexion réussie !")

Connexion réussie !


In [10]:
cur = conn.cursor()

# Lister toutes les tables du schéma public
cur.execute("""
SELECT table_name
FROM information_schema.tables
WHERE table_schema = 'public';
""")
tables = [t[0] for t in cur.fetchall()]

tables_columns = {}

for table in tables:
    cur.execute(f"""
    SELECT column_name
    FROM information_schema.columns
    WHERE table_name = '{table}';
    """)
    columns = [col[0] for col in cur.fetchall()]
    tables_columns[table] = columns

cur.close()
conn.close()

print(tables_columns)

{'hub_customers': ['customerid', 'loaddate', 'source'], 'link_customerinvoice': ['customerinvoiceid', 'customerid', 'invoiceid', 'loaddate', 'source'], 'hub_invoices': ['invoiceid', 'loaddate', 'source'], 'sat_customer': ['sat_customerid', 'customerid', 'customerfirstname', 'customerlastname', 'customerphone', 'customeremail', 'loaddate', 'source']}


In [33]:
now = datetime.now()

# HUB_CUSTOMERS
df_hub_customers = df[['CustomerId']].drop_duplicates().copy()
df_hub_customers['LoadDate'] = now
df_hub_customers['Source'] = 'daily_data'
df_hub_customers.columns = ['customerid', 'loaddate', 'source']

# HUB_INVOICES
df_hub_invoices = df[['InvoiceId']].drop_duplicates().copy()
df_hub_invoices['LoadDate'] = now
df_hub_invoices['Source'] = 'daily_data'
df_hub_invoices.columns = ['invoiceid', 'loaddate', 'source']

# LINK_CUSTOMERINVOICE
df_link_customerinvoice = df[['CustomerId', 'InvoiceId']].drop_duplicates().copy()
df_link_customerinvoice['LoadDate'] = now
df_link_customerinvoice['Source'] = 'daily_data'
df_link_customerinvoice.columns = ['customerid', 'invoiceid', 'loaddate', 'source']


# SAT_CUSTOMER
df_sat_customer = df[['CustomerId', 'CustomerFirstName', 'CustomerLastName', 'CustomerPhone', 'CustomerEmail']].drop_duplicates().copy()
df_sat_customer['LoadDate'] = now
df_sat_customer['Source'] = 'daily_data'
df_sat_customer.columns = ['customerid', 'customerfirstname', 'customerlastname', 'customerphone', 'customeremail', 'loaddate', 'source']


In [ ]:
conn = psycopg2.connect(**config)
cur = conn.cursor()

def insert_df(df, table_name):
    cols = ','.join(df.columns)
    values = [tuple(x) for x in df.to_numpy()]
    query = f"INSERT INTO {table_name} ({cols}) VALUES %s ON CONFLICT DO NOTHING"
    execute_values(cur, query, values)

# Chargement
# =========================
# HUBS
# =========================
insert_df(df_hub_customers, 'hub_customers')
# =========================
# LINKS
# =========================
insert_df(df_link_customerinvoice, 'link_customerinvoice')
# =========================
# SATELLITES
# =========================
insert_df(df_sat_customer, 'sat_customer')

conn.commit()
cur.close()
conn.close()

In [24]:
conn = psycopg2.connect(**config)

# Exemple pour hub_customers
df_hub = pd.read_sql("SELECT * FROM hub_customers;", conn)
print(df_hub.head(2))  # affiche les 10 premières lignes

# Idem pour sat_customer
df_sat = pd.read_sql("SELECT * FROM sat_customer;", conn)
print(df_sat.head(2))

# Et hub_invoices
df_inv = pd.read_sql("SELECT * FROM hub_invoices;", conn)
print(df_inv.head(2))

# Link_customerinvoice
df_link = pd.read_sql("SELECT * FROM link_customerinvoice;", conn)
print(df_link.head(2))

conn.close()

   customerid                   loaddate      source
0          21 2025-09-07 06:56:10.539957  daily_data
   sat_customerid  customerid customerfirstname customerlastname  \
0               1          21             Kathy            Chase   
1               2          21             Kathy            Chase   

       customerphone        customeremail                   loaddate  \
0  +1 (775) 223-7665  kachase@hotmail.com 2025-09-07 06:56:10.539957   
1  +1 (775) 223-7665  kachase@hotmail.com 2025-09-07 07:14:17.000000   

       source  
0  daily_data  
1  daily_data  
   invoiceid                   loaddate      source
0         16 2025-09-07 06:56:10.539957  daily_data
   customerinvoiceid  customerid  invoiceid                   loaddate  \
0                  1          21         16 2025-09-07 06:56:10.539957   
1                  2          21         16 2025-09-07 07:14:17.000000   

       source  
0  daily_data  
1  daily_data  


C:\Users\Sylla\AppData\Local\Temp\ipykernel_16220\3713328862.py:4: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_hub = pd.read_sql("SELECT * FROM hub_customers;", conn)
C:\Users\Sylla\AppData\Local\Temp\ipykernel_16220\3713328862.py:8: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_sat = pd.read_sql("SELECT * FROM sat_customer;", conn)
C:\Users\Sylla\AppData\Local\Temp\ipykernel_16220\3713328862.py:12: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_inv = pd.read_sql("SELECT * FROM hub_invoices;", conn)
C:\Users\Sylla